# Laboratorium 6 - Selekcja cech

### Importy

In [2]:
import scipy.io
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.feature_selection import RFE, SelectFromModel

### Wczytanie danych

In [3]:
data = scipy.io.loadmat('data/leukemia.mat')

X = data['X']
Y = data['Y'].ravel()

df_X = pd.DataFrame(X)
df_Y = pd.Series(Y)  

df = pd.concat([df_X, df_Y], axis=1)

df

,0,1,2,3,4,5,6,7,8,9,...,7061,7062,7063,7064,7065,7066,7067,7068,7069,0
0,-2,2,0,0,-2,-2,0,0,0,0,...,-2,-2,0,0,0,0,2,0,0,-1
1,0,0,0,0,-2,0,0,0,0,0,...,2,0,0,0,0,0,0,-2,0,-1
2,-2,-2,0,0,2,2,2,0,0,2,...,2,2,-2,2,2,2,2,0,0,-1
3,2,0,0,0,0,0,2,2,-2,2,...,2,2,-2,-2,0,-2,-2,0,-2,-1
4,-2,0,0,0,2,2,0,0,0,-2,...,0,2,0,0,-2,0,0,-2,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,0,-2,2,0,-2,2,-2,-2,0,...,0,0,0,-2,0,0,-2,0,2,1
68,0,0,-2,2,0,0,0,-2,0,-2,...,-2,2,0,2,-2,-2,0,2,0,1
69,2,2,0,2,2,0,2,2,2,2,...,0,0,0,0,0,2,0,0,2,1
70,0,-2,0,0,2,0,2,2,0,2,...,2,-2,0,-2,-2,0,0,0,-2,1


### Podział na zbiór treningowy i testowy

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=42)

### Usuń cechy z niską wariancją

In [5]:
threshold = 2.5  # im więcej, tym szybciej się liczy potem
selector = VarianceThreshold(threshold=threshold)
X_reduced = selector.fit_transform(df_X)
X_reduced.shape

(72, 1448)

### Podział na zbiór treningowy i testowy - usunięte cechy

In [6]:
X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X_reduced, df_Y, test_size=0.3, random_state=42)

In [7]:
n = X_train.shape[0]
m = n // 3

In [8]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
rfe_log_reg = RFE(log_reg, n_features_to_select=m)

In [9]:
rf_clf = RandomForestClassifier(random_state=42)
rfe_rf_clf = RFE(rf_clf, n_features_to_select=m)

### Zbiór z usuniętymi cechami

#### Regresja logistyczna

In [10]:
log_reg_cv_accuracy_reduced = cross_val_score(rfe_log_reg, X_train_reduced, y_train_reduced, cv=4, scoring='accuracy')
print(f"Średnia dokładność: {log_reg_cv_accuracy_reduced.mean():.4f}")

Średnia dokładność: 0.9199


In [11]:
log_reg_cv_auc_reduced = cross_val_score(rfe_log_reg, X_train_reduced, y_train_reduced, cv=4, scoring='roc_auc')
print(f"Średnia metryka AUC: {log_reg_cv_auc_reduced.mean():.4f}")

Średnia metryka AUC: 0.9929


#### Lasy losowe

In [15]:
rf_clf_cv_accuracy_reduced = cross_val_score(rf_clf, X_train_reduced, y_train_reduced, cv=4, scoring='accuracy')
print(f"Średnia dokładność: {rf_clf_cv_accuracy_reduced.mean():.4f}")

Średnia dokładność: 0.9391


In [16]:
rf_clf_cv_auc_reduced = cross_val_score(rf_clf, X_train_reduced, y_train_reduced, cv=4, scoring='roc_auc')
print(f"Średnia metryka AUC: {rf_clf_cv_auc_reduced.mean():.4f}")

Średnia metryka AUC: 0.9857


### Porównanie skuteczności

In [17]:

log_reg_l1 = LogisticRegression(penalty='l1', solver='liblinear', max_iter=1000, random_state=42)
selector_l1 = SelectFromModel(log_reg_l1)
X_l1_selected = selector_l1.fit_transform(X_train, y_train)

log_reg_cv_accuracy_l1 = cross_val_score(log_reg_l1, X_l1_selected, y_train, cv=4, scoring='accuracy')
print(f"Średnia dokładność (L1 - Logistic Regression): {log_reg_cv_accuracy_l1.mean():.4f}")

log_reg_cv_auc_l1 = cross_val_score(log_reg_l1, X_l1_selected, y_train, cv=4, scoring='roc_auc')
print(f"Średnia metryka AUC (L1 - Logistic Regression): {log_reg_cv_auc_l1.mean():.4f}")


Średnia dokładność (L1 - Logistic Regression): 0.9599
Średnia metryka AUC (L1 - Logistic Regression): 0.9938


In [19]:
rf_clf = RandomForestClassifier(random_state=42)
selector_rf = SelectFromModel(rf_clf)
X_rf_selected = selector_rf.fit_transform(X_train, y_train)

rf_clf_cv_accuracy = cross_val_score(rf_clf, X_rf_selected, y_train, cv=4, scoring='accuracy')
print(f"Średnia dokładność (Feature Importances - Random Forest): {rf_clf_cv_accuracy.mean():.4f}")

rf_clf_cv_auc = cross_val_score(rf_clf, X_rf_selected, y_train, cv=4, scoring='roc_auc')
print(f"Średnia metryka AUC (Feature Importances - Random Forest): {rf_clf_cv_auc.mean():.4f}")

Średnia dokładność (Feature Importances - Random Forest): 0.9792
Średnia metryka AUC (Feature Importances - Random Forest): 1.0000
